In [1]:
import numpy as np
import pandas as pd
import nupack as nu
import networkx as nx 
import matplotlib.pyplot as plt

from hdna.complex import Complex
from hdna.strand import Strand
from hdna.model import Model
from hdna.chamber import Chamber
from hdna.kinetwork import Kinetwork

from pyvis.network import Network

## About Julia-Python interface

In [3]:
import juliacall


jl = juliacall.newmodule('Dude')

jl.seval('using BioSimulator')
jl.seval('using TickTock')
jl.seval('using Base.Threads')

print(jl.nthreads())

jl.tick()
model = jl.Network("AT-all")  

initial = 2

model <= jl.Species("SS",int(initial))
model <= jl.Species("L1",0)
model <= jl.Species("L2",0)
model <= jl.Species("R1",0)
model <= jl.Species("R2",0)
model <= jl.Species("D",0)


kf_dl = 4e8
nonspecific_correction = 1/5

sliding_correction = 1/2
kf_sliding = 1e7 * sliding_correction



Na = 6.023e23

kf_ssl1 = kf_dl*nonspecific_correction
kb_ssl1 = 22.920792678077465 

kf_ssl2 = kf_dl*nonspecific_correction
kb_ssl2 = 0.007808225837362128

kf_ssr2 = kf_dl*nonspecific_correction
kb_ssr2 = 0.0626785751957368

kf_ssr1 = kf_dl*nonspecific_correction
kb_ssr1 = 183.99091641848813

kf_ssD = kf_dl*nonspecific_correction
kb_ssD = 6.902831840829526e-08

kf_l1l2 = kf_sliding
kb_l1l2 = 3406.6124793451313*sliding_correction

kf_l2D = kf_sliding
kb_l2D = 88.40461309148736*sliding_correction

kf_r1r2 = kf_sliding
kb_r1r2 = 3406.6124793451313*sliding_correction

kf_r2D = kf_sliding
kb_r2D = 11.01306438328072*sliding_correction




model <= jl.Reaction("ss-l1_f",  kf_ssl1, "SS + SS --> L1")
model <= jl.Reaction("ss-l1_b",  kb_ssl1, "L1 --> SS + SS")
model <= jl.Reaction("ss-l2_f",  kf_ssl2, "SS + SS --> L2")
model <= jl.Reaction("ss-l2_b",  kb_ssl2, "L2 --> SS + SS")
model <= jl.Reaction("ss-r2_f",  kf_ssr2, "SS + SS --> R2")
model <= jl.Reaction("ss-r2_b",  kb_ssr2, "R2 --> SS + SS")
model <= jl.Reaction("ss-r1_f",  kf_ssr1, "SS + SS --> R1")
model <= jl.Reaction("ss-r1_b",  kb_ssr1, "R1 --> SS + SS")
model <= jl.Reaction("ss-D_f",   kf_ssD, "SS + SS --> D")
model <= jl.Reaction("ss-D_b",   kb_ssD, "D --> SS + SS")
model <= jl.Reaction("l1-l2_f",  kf_l1l2, "L1 --> L2")
model <= jl.Reaction("l1-l2_b",  kb_l1l2, "L2 --> L1")
model <= jl.Reaction("l2-D_f",   kf_l2D, "L2 --> D")
model <= jl.Reaction("l2-D_b",   kb_l2D, "D --> L2")
model <= jl.Reaction("r1-r2_f",  kf_r1r2, "R1 --> R2")
model <= jl.Reaction("r1-r2_b",  kb_r1r2, "R2 --> R1")
model <= jl.Reaction("r2-D_f",   kf_r2D, "R2 --> D")
model <= jl.Reaction("r2-D_b",   kb_r2D, "D --> R2")



runtime = 1e-2
Nmonte = 50000
jl.tock()

jl.tick()
simresults = [jl.simulate(model, jl.Direct(),tfinal=runtime) for i in range(Nmonte)]
results = [{'c':jl.hcat(*simresults[i].u).__array__(), 't':simresults[i].t} for i in range(Nmonte)]
jl.tock()

1


[ Info:  started timer at: 2023-01-12T12:47:16.136


[ Model: AT-all ]
  no. species:    1
  no. reactions:  0

[ Model: AT-all ]
  no. species:    2
  no. reactions:  0

[ Model: AT-all ]
  no. species:    3
  no. reactions:  0

...

[ Model: AT-all ]
  no. species:    6
  no. reactions:  15

[ Model: AT-all ]
  no. species:    6
  no. reactions:  16

[ Model: AT-all ]
  no. species:    6
  no. reactions:  17

[ Model: AT-all ]
  no. species:    6
  no. reactions:  18

[ Info:            1.7258554s: 1 second, 725 milliseconds
[ Info:  started timer at: 2023-01-12T12:47:17.971
[ Info:            7.0182365s: 7 seconds, 18 milliseconds


In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
#definitive routine for mfpis is detailed below

index = 2

results[index]['c']
results[index]['c'][-1]
results[index]['t']

fpi = np.argmax(results[index]['c'][-1] == 1)

fpi, results[index]['t'][fpi]

array([[2, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 1, 1]])

array([0, 0, 1, 1])

4-element Vector{Float64}:
 0.0
 3.827390642420261e-9
 2.9768664821045267e-8
 0.01

(2, 2.9768664821045267e-08)

## About Chamber

In [3]:
model = Model('dna', '3D')
mincore = 3

a = Strand(model, 'AAAAAAAAAAA')
b = Strand(model, 'TTTTTTTTTTT')

kgraph = Kinetwork(model, a, b, mincore)
GR = kgraph.Graph

In [5]:
df = pd.DataFrame(list(pd.DataFrame(GR.nodes.data())[1]), index=pd.Series(GR.nodes()).index)
# for i in range(len(df)):
#     print(df.iloc[i])
#     print('\n')

for i in df.index:
    print(df.loc[i])
    print(df.loc[i]['object'].G)
    print('\n')

object                          None
structure    ...........+...........
state                 singlestranded
pairs                              0
Name: ...........+..........., dtype: object


AttributeError: 'NoneType' object has no attribute 'G'